# Many Species Population Dynamics

### Q' inich Figueroa Coc: Gruppe 5
### Paris J. Huth: Gruppe 1

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 

## 1.
For one there is the trivial fixed point of $n_0=0$. In the case of the Volterra-Lotka System there is also the fixed point of $n_1=1$.

For the third fp we examin the growth limiting factor, thus considering the prey population dynamic without predators:

$$ \frac{\mathrm{d}N_i}{\mathrm{d}t}=N_i\left(a_i-N_i\right)$$

Which yields: $n_2 = \left(11,12,10\right)$.

## 2.

In [28]:
def dNN(n,a,b,p):
    N = []
    Ntemp = []
    for i in range(0,3):
        n[i] = a[i]-2*n[i]

        for j in range(0,3):
            Ntemp.append(n[i]-b[i,j]*p[j])
        N.append(Ntemp)
    return(np.array(N))

def dNP(n,a,b,p):
    N = []
    Ntemp = []
    for i in range(0,3):
        for j in range(0,3):
            Ntemp.append(-n[i]*b[i,j])
        N.append(Ntemp)
    return(np.array(N))

def dPP(n,c,d,p):
    P = []
    Ptemp = []
    for i in range(0,3):
        for j in range(0,3):
            Ptemp.append(c[i,j]*n[j]-d[i])
        P.append(Ptemp)
    return(np.array(P))

def dPN(n,c,d,p):
    P = []
    Ptempt = []
    for i in range(0,3):
        for j in range(0,3):
            Ptemp.append(p[i]*[i,j])
        P.append(Ptemp) 
    return(np.array(P))

In [29]:
a = np.array([11,12,10])
d = np.array([8,9,17])
b = np.array([[2,3,5],[1,3,7],[4,3,2]])
c = np.array([[2,3,3],[3,3,3],[7,8,2]])

In [31]:
n1 = np.ones(3)
p1 = np.ones(3)
dNN(n1,a,b, p1)
dNP(n1,a,b,p1)

array([[-18., -27., -45., -10., -30., -70., -32., -24., -16.],
       [-18., -27., -45., -10., -30., -70., -32., -24., -16.],
       [-18., -27., -45., -10., -30., -70., -32., -24., -16.]])